**NLP Ner Assingment**


In [ ]:
!curl -LJO https://raw.githubusercontent.com/SilentFlame/Named-Entity-Recognition/master/Twitterdata/annotatedData.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1554k  100 1554k    0     0  1794k      0 --:--:-- --:--:-- --:--:-- 1792k


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Iterable

In [ ]:
data=pd.read_csv('/content/annotatedData.csv')

In [ ]:
data.head()

,Sent,Word,Tag
0,sent: 0,Bahubali,B-Per
1,sent: 0,2,Other
2,sent: 0,ke,Other
3,sent: 0,liye,Other
4,sent: 0,line,Other


In [ ]:
data['Word']=data['Word'].astype('str')

In [ ]:
text=data.Word.to_list()

In [ ]:
for_vocab=""
for i in text:
  for_vocab+=i

In [ ]:
tags=data.Tag.to_list()

In [ ]:
#Paper method
def boi(k,v):
  if v == "B-Per" or v == "I-Per":
    # print('w')
    n=len(k)
    lst=['P']*n
    val=''.join(lst)
    return lst
  elif v=='B-Org' or v=='I-Org':
    lst=['G']*len(k)
    # lst[0]='G'
    # lst[-1]='G'
    val=''.join(lst)
    return lst

  elif v=='B-Loc' or v=='I-Loc':
    lst=['L']*len(k)
    # lst[0]='L'
    # lst[-1]='L'
    val=''.join(lst)
    return lst
  elif v=='Other':
    lst=['O']*len(k)
    val=''.join(lst)
    return lst


In [ ]:
tag_dic={'P':1,'G':2,'L':3,'O':0}

In [ ]:
rev_tag_dic={1:'P',2:'G',3:'L',0:'O'}

In [ ]:
def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:        
             yield item

In [ ]:
# The unique characters in the file
vocab = sorted(set(for_vocab))
print('{} unique characters'.format(len(vocab)))

108 unique characters


In [ ]:
char2idx = dict((c, i) for i, c in enumerate(vocab))

In [ ]:
idx2char = np.array(vocab)

In [ ]:
inter=data

In [ ]:
def word_to_char(x):
  lst=[i for i in x]
  lst.append(" ")
  return lst

def char_to_idx(x):
  return [char2idx[i] for i in x]
def tag_to_idx(x):
  return [tag_dic[i] for i in x]

In [ ]:
inter['Word']=inter['Word'].apply(lambda x:word_to_char(x))

In [ ]:
inter.head()

,Sent,Word,Tag
0,sent: 0,"[B, a, h, u, b, a, l, i, ]",B-Per
1,sent: 0,"[2, ]",Other
2,sent: 0,"[k, e, ]",Other
3,sent: 0,"[l, i, y, e, ]",Other
4,sent: 0,"[l, i, n, e, ]",Other


In [ ]:
# inter['Tag']=inter['Tag'].apply(boi(inter['Word'],inter['Tag']),axis=1)
y=[]
for index, row in tqdm(inter.iterrows()):
  y.append(boi(row['Word'],row['Tag']))


72143it [00:05, 12208.36it/s]


In this each character of a word is tagged with the tag of the word i.e
say we have a enity jim which is B-Per so each character of this word will have a label P so labels for jim will be [P,P,P]

In [ ]:
inter['BOI']=y

In [ ]:
inter_dropna=inter.dropna()

In [ ]:
inter_dropna.head()

,Sent,Word,Tag,BOI
0,sent: 0,"[B, a, h, u, b, a, l, i, ]",B-Per,"[P, P, P, P, P, P, P, P, P]"
1,sent: 0,"[2, ]",Other,"[O, O]"
2,sent: 0,"[k, e, ]",Other,"[O, O, O]"
3,sent: 0,"[l, i, y, e, ]",Other,"[O, O, O, O, O]"
4,sent: 0,"[l, i, n, e, ]",Other,"[O, O, O, O, O]"


In [ ]:
data_group = inter_dropna.groupby(
['Sent'],as_index=False
)['Word', 'Tag', 'BOI'].agg(lambda x: list(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data_group.head()


,Sent,Word,Tag,BOI
0,sent: 0,"[[B, a, h, u, b, a, l, i, ], [2, ], [k, e, ...","[B-Per, Other, Other, Other, Other, Other, Oth...","[[P, P, P, P, P, P, P, P, P], [O, O], [O, O, O..."
1,sent: 1,"[[a, g, a, r, ], [#, n, o, t, e, b, a, n, d, ...","[Other, Other, Other, Other, B-Org, I-Org, Oth...","[[O, O, O, O, O], [O, O, O, O, O, O, O, O, O, ..."
2,sent: 10,"[[#, n, o, t, e, b, a, n, d, i, ], [k, e, ],...","[Other, Other, Other, Other, Other, B-Org, Oth...","[[O, O, O, O, O, O, O, O, O, O, O], [O, O, O],..."
3,sent: 100,"[[M, a, s, t, e, r, s, t, r, o, k, e, ], [t, ...","[Other, Other, B-Per, Other, Other, Other, Oth...","[[O, O, O, O, O, O, O, O, O, O, O, O, O], [O, ..."
4,sent: 1000,"[[J, i, n, ], [l, o, g, o, n, ], [k, o, ], ...","[Other, Other, Other, B-Org, Other, B-Per, Oth...","[[O, O, O, O], [O, O, O, O, O, O], [O, O, O], ..."


In [ ]:
data_group['Word']=data_group['Word'].apply(flatten).apply(list)
data_group['BOI']=data_group['BOI'].apply(flatten).apply(list)

In [ ]:
data_group['char_idx']=data_group['Word'].apply(char_to_idx)
data_group['tag_idx']=data_group['BOI'].apply(tag_to_idx)

In [ ]:
data_group.head()

,Sent,Word,Tag,BOI,char_idx,tag_idx
0,sent: 0,"[B, a, h, u, b, a, l, i, , 2, , k, e, , l, ...","[B-Per, Other, Other, Other, Other, Other, Oth...","[P, P, P, P, P, P, P, P, P, O, O, O, O, O, O, ...","[32, 58, 65, 78, 59, 58, 69, 66, 0, 17, 0, 68,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ..."
1,sent: 1,"[a, g, a, r, , #, n, o, t, e, b, a, n, d, i, ...","[Other, Other, Other, Other, B-Org, I-Org, Oth...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[58, 64, 58, 75, 0, 2, 71, 72, 77, 62, 59, 58,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,sent: 10,"[#, n, o, t, e, b, a, n, d, i, , k, e, , b, ...","[Other, Other, Other, Other, Other, B-Org, Oth...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 71, 72, 77, 62, 59, 58, 71, 61, 66, 0, 68,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,sent: 100,"[M, a, s, t, e, r, s, t, r, o, k, e, , t, o, ...","[Other, Other, B-Per, Other, Other, Other, Oth...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[43, 58, 76, 77, 62, 75, 76, 77, 75, 72, 68, 6...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,sent: 1000,"[J, i, n, , l, o, g, o, n, , k, o, , d, e, ...","[Other, Other, Other, B-Org, Other, B-Per, Oth...","[O, O, O, O, O, O, O, O, O, O, O, O, O, G, G, ...","[40, 66, 71, 0, 69, 72, 64, 72, 71, 0, 68, 72,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, ..."


In [ ]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
def get_pad_train_test_val(data_group):

    #Pad tokens (X var)    
    tokens = data_group['char_idx'].tolist()
    maxlen =max_len=max([len(i) for i in data_group['Word'].to_list()])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post')

    #Pad Tags (y var) and convert it into one hot encoding
    tags = data_group['tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag_dic['O'])
    n_tags = len(tag_dic)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]

    #Split train, test and validation set
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.2, train_size=0.8, random_state=2020)
    # train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)


    print("train_data_x ->",tokens_.shape)
    print("teset_data_x ->",test_tokens.shape)
    print("train_data_y ->",len(tags_))
    print("test_data_y ->",len(test_tags))


    return tokens_, test_tokens, tags_, test_tags

In [ ]:
train_data_x,test_data_x,train_data_y,test_data_y=get_pad_train_test_val(data_group)

train_data_x -> (2468, 662)
teset_data_x -> (617, 662)
train_data_y -> 2468
test_data_y -> 617


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding, TimeDistributed, Dropout,Bidirectional,Conv1D,MaxPooling1D,Flatten,Input


In [ ]:
#Paper 
model = Sequential()
model.add(Embedding(input_dim=len(vocab), output_dim=64,input_length=662))
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2,)))
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(TimeDistributed(Dense(4, activation='softmax'))) #change 3 to 5 for predicting entitites
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
# print(model)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 662, 64)           6912      
_________________________________________________________________
bidirectional (Bidirectional (None, 662, 256)          197632    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 662, 256)          394240    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 662, 256)          394240    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 662, 256)          394240    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 662, 256)          394240    
_________________________________________________________________
dropout (Dropout)            (None, 662, 256)          0

In [ ]:
X = np.reshape(train_data_x, (train_data_x.shape[0], train_data_x.shape[1],1))
# X = np.reshape(train_data_x, (train_data_x.shape[0],60,662,1))

In [ ]:
y=np.array(train_data_y)

In [ ]:
# y=np.squeeze(y,axis=1)
y.shape

(2468, 662, 4)

In [ ]:
X.shape

(2468, 662, 1)

In [ ]:
model.fit(train_data_x,y,epochs=10, batch_size=32, validation_split = 0.2,verbose=1)

Epoch 1/10
62/62 [==============================] - 43s 413ms/step - loss: 0.3003 - accuracy: 0.9191 - val_loss: 0.0766 - val_accuracy: 0.9809
Epoch 2/10
62/62 [==============================] - 23s 368ms/step - loss: 0.0696 - accuracy: 0.9819 - val_loss: 0.0626 - val_accuracy: 0.9823
Epoch 3/10
62/62 [==============================] - 23s 368ms/step - loss: 0.0649 - accuracy: 0.9820 - val_loss: 0.0609 - val_accuracy: 0.9827
Epoch 4/10
62/62 [==============================] - 23s 368ms/step - loss: 0.0604 - accuracy: 0.9833 - val_loss: 0.0622 - val_accuracy: 0.9827
Epoch 5/10
62/62 [==============================] - 23s 368ms/step - loss: 0.0591 - accuracy: 0.9837 - val_loss: 0.0589 - val_accuracy: 0.9832
Epoch 6/10
62/62 [==============================] - 23s 368ms/step - loss: 0.0603 - accuracy: 0.9830 - val_loss: 0.0570 - val_accuracy: 0.9840
Epoch 7/10
62/62 [==============================] - 23s 368ms/step - loss: 0.0574 - accuracy: 0.9835 - val_loss: 0.0556 - val_accuracy: 0.9843

All these accuracies that are displayed here are charater level word level accuracy on test set is calculated below using the function test()

In [ ]:
test = np.reshape(test_data_x, (test_data_x.shape[0], test_data_x.shape[1],1))
test_y=np.array(test_data_y)

In [ ]:
model.evaluate(test,test_y,batch_size=32) #so this is again char level accuracy on test set

20/20 [==============================] - 5s 128ms/step - loss: 0.0515 - accuracy: 0.9850


[0.05154358595609665, 0.9850117564201355]

In [ ]:
def generate_pred_string(prediction):


  pred_string=""
  #this function does temprature sampling
  def sample(preds, temperature=1.0):
      # helper function to sample an index from a probability array
      preds = np.asarray(preds).astype('float64')
      preds = np.log(preds) / temperature
      exp_preds = np.exp(preds)
      preds = exp_preds / np.sum(exp_preds)
      probas = np.random.multinomial(1, preds, 1)
      return np.argmax(probas)
  #this function is for normal sampling 
  def samplet(prediction):
    # idx=np.random.choice(4,p=p[0])
    # idx=np.random.multinomial(1,p[0])
    idx=np.argmax(p)  #This gives best results
    return idx
  for p in prediction:
    #0.8 was good
    # pred_string+=rev_tag_dic[sample(p[0],temperature=0.8)]
    pred_string+=rev_tag_dic[samplet(p[0])]
  return pred_string


In [ ]:
from collections import Counter
def convert_to_word_level(zip_tag_preds):
  lst=[]
  temp=""
  for i in zip_tag_preds:
    if (i[0]==' '):
      if len(temp)==0:
        lst.append('O')
      else:
        res=Counter(temp)
        chr=max(res,key=res.get)
        # print(chr)
        temp=""
        lst.append(chr)
        # lst.append('O')
    else:
      temp+=i[1]
  return lst


**Word level Model evaluation on test set**

Research paper referred  https://www.aclweb.org/anthology/C16-1087.pdf

so converting char level predictions to word level is done using majority voting(see the mentioned paper)
breifly say we have a word Thomas and we have charater level predictions for this like OPPLGP as we can see most probable tag(max occuring) is P but it is not consistent so we will use majority voting for this which we will give Thomas a tag of P.
In paper first they used majority voting then they used more sophisticated approach which is viterbi decoder(used this to assign word level tags)

In [ ]:
#Use majority voting to convert char level tags to word level and calculate accuracy
from sklearn.metrics import accuracy_score
from tqdm import tqdm
def test():
  acc=0
  count=len(test_data_x)
  for t in tqdm(range(len(test_data_x))):
    # print(t)
    lst_text=test_data_x[t]
    # hm=662-len(lst_text)
    # lst_text=lst_text+[0]*hm
    ln=np.nonzero(lst_text)
    # print(ln)
    ln=np.max(ln)
    prediction=model.predict(lst_text)
    pred_string=generate_pred_string(prediction)
    tr_pred_string=""
    for i in test_data_y[t]:
      tr_pred_string+=rev_tag_dic[np.argmax(i)]
    tex=get_text(lst_text)
    # print(tex)
    tex=tex[:ln+1]
    # print(len(tex.split(' ')))
    z=list(zip(tex,pred_string[:ln+1]))
    z_tr=list(zip(tex,tr_pred_string[:ln+1]))
    preds=convert_to_word_level(z)
    trpreds=convert_to_word_level(z_tr)
    # trpreds=trpreds[:ln+1]
    # print(preds)
    # print(trpreds)
    if len(preds)==0 and len(trpreds)==0:
      count-=1
      continue
    # print(accuracy_score(trpreds,preds))
    acc=acc+accuracy_score(trpreds,preds)
    # print(acc)
    # break
  # print(acc)
  print("")
  print("Word level accuracy on test set is :-")
  print((acc*100)/count)
  # print(acc/test_data_x.shape[0])
  # return accuracy_score(trpreds,gpreds)


In [ ]:
test()



  0%|          | 0/617 [00:00<?, ?it/s]

  0%|          | 1/617 [00:00<01:54,  5.38it/s]

  0%|          | 2/617 [00:00<01:54,  5.38it/s]

  0%|          | 3/617 [00:00<01:51,  5.52it/s]

  1%|          | 4/617 [00:00<01:49,  5.58it/s]

  1%|          | 5/617 [00:00<01:48,  5.64it/s]

  1%|          | 6/617 [00:01<01:48,  5.62it/s]

  1%|          | 7/617 [00:01<01:47,  5.65it/s]

  1%|▏         | 8/617 [00:01<01:47,  5.67it/s]

  1%|▏         | 9/617 [00:01<01:46,  5.73it/s]

  2%|▏         | 10/617 [00:01<01:47,  5.67it/s]

  2%|▏         | 11/617 [00:01<01:46,  5.67it/s]

  2%|▏         | 12/617 [00:02<01:47,  5.63it/s]

  2%|▏         | 13/617 [00:02<01:46,  5.65it/s]

  2%|▏         | 14/617 [00:02<01:45,  5.72it/s]

  2%|▏         | 15/617 [00:02<01:44,  5.77it/s]

  3%|▎         | 16/617 [00:02<01:45,  5.70it/s]

  3%|▎         | 17/617 [00:02<01:44,  5.73it/s]

  3%|▎         | 18/617 [00:03<01:46,  5.64it/s]

  3%|▎         | 19/617 [00:03<01:46,  5.61it/s]

  3%|▎         |


Word level accuracy on test set is :-
92.36141079527685


Final word level accuracy

92.36141079527685